In [1]:
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Span

output_notebook()


Loading BokehJS ...

In [2]:
# reload 

from importlib import reload

import IntLib
import AutoDiff
import GenFuncs

reload(IntLib)
reload(AutoDiff)
reload(GenFuncs)


<module 'GenFuncs' from 'c:\\Users\\k1m19\\OneDrive\\Desktop\\Numerical\\IntLib\\GenFuncs.py'>

In [3]:
from IntLib import Interval as IA

from AutoDiff import (
    AutoDiff as AD,
    _ensure_AD as ensure_AD,
)

from GenFuncs import (
    mag,
    mig,
    sqrt,
    sin,
    cos,
    mid,
    rad,
    exp,
    log,
    atan,
)



# TODO
- tests
- deal with empty set
- extened division ?
- numpy arrays...

# Newton method for root finding

In [15]:
def f(x):
    return sin(exp(x) + 1)

x = np.linspace(0, 3, 1_000)

fig = figure(width=600, height=300)
fig.line(x, f(x))

show(fig)

# Newton on ℝ

In [5]:
def f(x):
    return sin(exp(x) + 1)

def newton_step(f, x: AD):
    xx = ensure_AD(x, 1)
    fx = f(xx)
    return x - fx.x / fx.dx


def newton(f, x, tol, dbg=False):
    y = newton_step(f, x)
    i = 0
    acc = []
    while abs(y - x) > tol:
        if dbg:
            print(f"{i}: x={x}")
        x, y = y, newton_step(f, y)
        acc.append(x)
        i += 1
    return y, acc


newton(f, 1, 1e-10)[0], newton(f, 0, 1e-10)[0]

(np.float64(0.7615497828808945), np.float64(2.13117712108631))

# IA Newton

In [9]:
f = lambda x: 3 * x - x**3

x = np.linspace(0, 3, 10_000)

fig = figure(width=600, height=300)
fig.line(x, f(x))

show(fig)

In [11]:
def newton_step_IA(f, x):
    xx = ensure_AD(x, 1)  # variable
    fx = f(xx)
    mx = IA(mid(x))
    nx = mx - f(mx) / fx.dx
    print(x, nx)
    return x & nx


print_int = lambda x: print(f"{x} {x.rad:+g}")

def newton_IA(f, x, tol):
    x0 = x
    while rad(x) * 2 > tol:
        print_int(x)
        x = newton_step_IA(f, x)
        print_int(x)
        if x.is_extended:
            print("extended interval")
            return
        if x.isempty:
            print("no roots")
            return
    if x in x0:
        print('unique root')


f = lambda x: -2.001 + 3 * x - x**3

# TODO broken
# f = lambda x: 3 * x - x**3

newton_IA(f, IA(-3, 3), 0.01)

[-3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀] +3
[-3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀, +3.₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀₀] [-∞, -₀.₀83375₀₀₀₀₀₀₀₀₀₀₀]∪[+₀.66699999999999993, +∞]
[-∞, -₀.₀83375₀₀₀₀₀₀₀₀₀₀₀]∪[+₀.66699999999999993, +∞] -0.375187
extended interval


In [7]:
# TODO: 
# can this be done without classes?
# incorporate both AD and IA